In [1]:
# Import Dependencies

import pandas as pd

import gmaps

import requests

#Import the API key

from config import g_key

In [2]:
# Store the csv you saved created in part one into a Data Frame

city_data_df = pd.read_csv("weather_data/cities.csv")

city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2022-10-11 02:07:56,-23.1203,-134.9692,72.00,72,47,1.19
1,1,Srednekolymsk,RU,2022-10-11 02:07:57,67.4500,153.6833,24.44,73,41,14.41
2,2,Macia,MZ,2022-10-11 02:07:57,-25.0269,33.0989,68.02,94,0,0.92
3,3,Zhigansk,RU,2022-10-11 02:07:58,66.7697,123.3711,19.51,87,21,8.43
4,4,East London,ZA,2022-10-11 02:07:58,-33.0153,27.9116,68.59,81,100,15.17


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key

gmaps.configure(api_key=g_key)

In [5]:
# Get max temps without negatives

max_temp = city_data_df["Max Temp"]

temps = []


In [6]:
# Create a heatmap for max temp
# Assign the locations to an array of latitude and longitude pairs.

locations = city_data_df[["Lat", "Lng"]]

#Assign the weights variable to some values. 

max_temp = city_data_df["Max Temp"]

# Assign the figure variable to the gmaps.figure() attribute.

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations. 

heatmap_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)


# Add the heatmap layer

fig.add_layer(heatmap_layer)

# Call the figure to plot the data

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity

locations = city_data_df[["Lat", "Lng"]]

humidity = city_data_df["Humidity"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent Cloudiness

locations = city_data_df[["Lat", "Lng"]]

clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of wind mph

locations = city_data_df[["Lat", "Lng"]]

wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the custoer to add a minimum and maximum temperature value.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.count()

City_ID       158
City          158
Country       158
Date          158
Lat           158
Lng           158
Max Temp      158
Humidity      158
Cloudiness    158
Wind Speed    158
dtype: int64

In [12]:
# Create a DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates. 

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

hotel_df["Hotel Name"] = ""

hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Cabo Rojo,PR,79.97,18.0866,-67.1457,
8,Lorengau,PG,85.10,-2.0226,147.2712,
11,Ambulu,ID,81.18,-8.3450,113.6058,
23,Kavieng,PG,84.90,-2.5744,150.7967,
26,Hilo,US,84.27,19.7297,-155.0900,
31,Waingapu,ID,87.28,-9.6567,120.2641,
39,Pangai,TO,76.15,-19.8000,-174.3500,
41,Kampong Thum,KH,76.68,12.7111,104.8887,
46,Kapaa,US,84.18,22.0752,-159.3190,
48,Nago,JP,75.63,26.5881,127.9761,


In [13]:
# Set parameters to search for a hotel

import requests

import gmaps

from config import g_key

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [28]:
# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,                                 
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))